In [1]:
import pandas as pd
import numpy as np
import string  
import nltk
import re
import scipy
import matplotlib.pyplot as plt
import urllib 
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error, f1_score

!pip install catboost 
from catboost import CatBoostRegressor, CatBoostClassifier
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

display(train)
display(test)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [2]:
train['keyword'].fillna('no keyword', inplace = True) 
train['keyword'] = train['keyword'].str.replace('%20', ' ')
train['location'].fillna('no location', inplace = True)

test['keyword'].fillna('no keyword', inplace = True) 
test['keyword'] = test['keyword'].str.replace('%20', ' ')
test['location'].fillna('no location', inplace = True)

train['length'] = train['text'].str.len()

test['length'] = test['text'].str.len()

train.head()

,id,keyword,location,text,target,length
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88


In [3]:
#limpiando el texto

import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

#Cleaning text

train['clean_text'] = train['text'].str.lower()

train['clean_text'] = train['clean_text'].apply(only_letters)    

test['clean_text'] = test['text'].str.lower()

test['clean_text'] = test['clean_text'].apply(only_letters) 

#Tokenización

train['clean_text'] = train['clean_text'].apply(word_tokenize)

test['clean_text'] = test['clean_text'].apply(word_tokenize)

#Remove stopwords

train['clean_text'] = train['clean_text'].apply(filter_stopwords) 

test['clean_text'] = test['clean_text'].apply(filter_stopwords) 


#Lemmatization                                                                       

train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  

train['clean_text'] = train['clean_text'].apply(transform_to_text)

test['clean_text'] = test['clean_text'].apply(lemmatize_tweet)  

test['clean_text'] = test['clean_text'].apply(transform_to_text)

display(train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,keyword,location,text,target,length,clean_text
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69,deed reason earthquake may allah forgive u
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38,forest fire near la ronge sask canada
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133,resident asked shelter place notified officer ...
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65,people receive wildfire evacuation order calif...
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88,got sent photo ruby alaska smoke wildfire pour...
...,...,...,...,...,...,...,...
7608,10869,no keyword,no location,Two giant cranes holding a bridge collapse int...,1,83,two giant crane holding bridge collapse nearby...
7609,10870,no keyword,no location,@aria_ahrary @TheTawniest The out of control w...,1,125,ariaahrary thetawniest control wild fire calif...
7610,10871,no keyword,no location,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,utckm volcano hawaii
7611,10872,no keyword,no location,Police investigating after an e-bike collided ...,1,137,police investigating ebike collided car little...


In [4]:
#agregando algunos features

keywords_list = list(set(train.keyword))  #todas las keywords únicas

locations_list = list(set(train.location)) #todas las locaciones únicas
locations_list.remove('no location')

def amount_keywords(tweet):
    cont = 0
    for keyword in keywords_list:
        if keyword in tweet:
            cont += 1
    return cont

train['amount_words'] = train['clean_text'].str.split(' ').apply(lambda x: len(x))  

test['amount_words'] = test['clean_text'].str.split(' ').apply(lambda x: len(x))  

#conviene que cuente del tweet limpio o del tweet original??

train['amount_keywords_in_tweet'] = train['clean_text'].apply(lambda x: amount_keywords(x))

test['amount_keywords_in_tweet'] = test['clean_text'].apply(lambda x: amount_keywords(x))


tiene_mas_de_un_kw = train[train['amount_keywords_in_tweet'] > 1]

tiene_un_solo_kw = train[train['amount_keywords_in_tweet'] == 1]

no_tiene_kw = train[train['amount_keywords_in_tweet'] == 0]

porcentaje_target_mas_de_un_kw = tiene_mas_de_un_kw['target'].mean()
print("porcentaje target de tweets con mas de un kw", porcentaje_target_mas_de_un_kw)

porcentaje_target_un_solo_kw = tiene_un_solo_kw['target'].mean()
print("porcentaje target de tweets con un solo kw", porcentaje_target_un_solo_kw)

porcentaje_target_sin_kw = no_tiene_kw['target'].mean()
print("porcentaje target de tweets sin kw", porcentaje_target_sin_kw)

print("el maximo de cant de keywords en un tweet es", tiene_mas_de_un_kw['amount_keywords_in_tweet'].max())

def amount_locations(tweet):    #esta función podría unificarse con la de keywords pasandole por parámetro la lista a recorrer
    cont = 0
    for location in locations_list:
        if location in tweet:
            cont += 1
    return cont

train['amount_locations_in_tweet'] = train['clean_text'].apply(lambda x: amount_locations(x))

test['amount_locations_in_tweet'] = test['clean_text'].apply(lambda x: amount_locations(x))


mas_de_1_location = train[train['amount_locations_in_tweet'] > 1]

un_solo_location = train[train['amount_locations_in_tweet'] == 1]

no_tiene_location = train[train['amount_locations_in_tweet'] == 0]

porcentaje_target_mas_de_un_location = mas_de_1_location['target'].mean()
print("porcentaje target de tweets con mas de una locacion", porcentaje_target_mas_de_un_location)

porcentaje_target_un_solo_location = un_solo_location['target'].mean()
print("porcentaje target de tweets con una sola locacion", porcentaje_target_un_solo_location)

porcentaje_target_sin_location = no_tiene_location['target'].mean()
print("porcentaje target de tweets sin locacion", porcentaje_target_sin_location)

print("el maximo de cant de locaciones en un tweet es", mas_de_1_location['amount_locations_in_tweet'].max())

porcentaje target de tweets con mas de un kw 0.606816584680253
porcentaje target de tweets con un solo kw 0.3446327683615819
porcentaje target de tweets sin kw 0.2025862068965517
el maximo de cant de keywords en un tweet es 7
porcentaje target de tweets con mas de una locacion 0.504933504933505
porcentaje target de tweets con una sola locacion 0.4489409984871407
porcentaje target de tweets sin locacion 0.34382107657316147
el maximo de cant de locaciones en un tweet es 8


In [5]:
mas_de_un_location_y_mas_de_un_keyword = train[(train['amount_locations_in_tweet'] > 1) & (train['amount_keywords_in_tweet'] > 1)]
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword = mas_de_un_location_y_mas_de_un_keyword.target.mean()
porcentaje_target_mas_de_un_location_y_mas_de_un_keyword

0.6990196078431372

In [6]:
#más features
def get_hashtags(s):
    return list(part[1:] for part in s.split() if part.startswith('#'))

train['hashtags'] = train['text'].apply(get_hashtags)
test['hashtags'] = test['text'].apply(get_hashtags)

def hashtag_length_proportion(hashtags, length):
    return len(''.join(hashtags))/length

#VER SI DEJAR ESTEEE
train['len_hash_over_text'] = train.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)
test['len_hash_over_text'] = test.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)

def get_mentioned_users(s):
    return list(part[1:] for part in s.split() if part.startswith('@'))

train['users'] = train['text'].apply(get_mentioned_users)
test['users'] = test['text'].apply(get_mentioned_users)

def amount_hashtags(l):
    return len(l)

train['amount_hashtags'] = train['hashtags'].apply(amount_hashtags)
test['amount_hashtags'] = test['hashtags'].apply(amount_hashtags)

def amount_users(l):
    return len(l)

def has_url(text):
    return int('http' in text)

train['has_url'] = train['text'].apply(has_url)
test['has_url'] = test['text'].apply(has_url)

def find_url(text):
    urls = re.findall(r'(https?://\S+)', text)
    return urls

train['urls'] = train['text'].apply(find_url)
test['urls'] = test['text'].apply(find_url)
train['amount_urls'] = train['urls'].apply(lambda x: len(x))
test['amount_urls'] = test['urls'].apply(lambda x: len(x))

train['amount_users'] = train['users'].apply(amount_users)
test['amount_users'] = test['users'].apply(amount_users)


In [7]:
#construyendo los datos de entrenamiento y de test
selected_df = train[['length','amount_words','amount_keywords_in_tweet','amount_locations_in_tweet', 'amount_urls','amount_users', 'amount_hashtags','len_hash_over_text', 'has_url']] 
#ver después de codificar las variables categóricas para agregarlas
X, y = selected_df,train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [8]:
#Catboost como problema de regresión
model = CatBoostRegressor(random_seed = 1)
model.fit(X_train, y_train)
preds = model.predict(X_test)

Learning rate set to 0.053835
0:	learn: 0.4909842	total: 388ms	remaining: 6m 27s
1:	learn: 0.4873706	total: 417ms	remaining: 3m 27s
2:	learn: 0.4844860	total: 422ms	remaining: 2m 20s
3:	learn: 0.4816724	total: 427ms	remaining: 1m 46s
4:	learn: 0.4787181	total: 434ms	remaining: 1m 26s
5:	learn: 0.4762516	total: 441ms	remaining: 1m 13s
6:	learn: 0.4737690	total: 474ms	remaining: 1m 7s
7:	learn: 0.4713176	total: 506ms	remaining: 1m 2s
8:	learn: 0.4691991	total: 514ms	remaining: 56.6s
9:	learn: 0.4672028	total: 524ms	remaining: 51.9s
10:	learn: 0.4655222	total: 531ms	remaining: 47.7s
11:	learn: 0.4639285	total: 557ms	remaining: 45.9s
12:	learn: 0.4625098	total: 568ms	remaining: 43.1s
13:	learn: 0.4612837	total: 601ms	remaining: 42.3s
14:	learn: 0.4602475	total: 605ms	remaining: 39.7s
15:	learn: 0.4590573	total: 614ms	remaining: 37.7s
16:	learn: 0.4580871	total: 690ms	remaining: 39.9s
17:	learn: 0.4570775	total: 740ms	remaining: 40.4s
18:	learn: 0.4562106	total: 746ms	remaining: 38.5s
19:	l

163:	learn: 0.4319748	total: 3.07s	remaining: 15.7s
164:	learn: 0.4319228	total: 3.08s	remaining: 15.6s
165:	learn: 0.4318352	total: 3.09s	remaining: 15.5s
166:	learn: 0.4317868	total: 3.1s	remaining: 15.4s
167:	learn: 0.4317080	total: 3.14s	remaining: 15.6s
168:	learn: 0.4316322	total: 3.16s	remaining: 15.6s
169:	learn: 0.4315652	total: 3.21s	remaining: 15.7s
170:	learn: 0.4314469	total: 3.23s	remaining: 15.7s
171:	learn: 0.4313262	total: 3.27s	remaining: 15.7s
172:	learn: 0.4312908	total: 3.28s	remaining: 15.7s
173:	learn: 0.4312099	total: 3.29s	remaining: 15.6s
174:	learn: 0.4310698	total: 3.29s	remaining: 15.5s
175:	learn: 0.4310151	total: 3.3s	remaining: 15.5s
176:	learn: 0.4309154	total: 3.31s	remaining: 15.4s
177:	learn: 0.4308856	total: 3.36s	remaining: 15.5s
178:	learn: 0.4307954	total: 3.4s	remaining: 15.6s
179:	learn: 0.4306769	total: 3.41s	remaining: 15.5s
180:	learn: 0.4304551	total: 3.41s	remaining: 15.4s
181:	learn: 0.4303494	total: 3.42s	remaining: 15.4s
182:	learn: 0.4

324:	learn: 0.4199505	total: 5.73s	remaining: 11.9s
325:	learn: 0.4198612	total: 5.78s	remaining: 11.9s
326:	learn: 0.4198368	total: 5.83s	remaining: 12s
327:	learn: 0.4197419	total: 5.84s	remaining: 12s
328:	learn: 0.4196778	total: 5.85s	remaining: 11.9s
329:	learn: 0.4196322	total: 5.85s	remaining: 11.9s
330:	learn: 0.4195932	total: 5.87s	remaining: 11.9s
331:	learn: 0.4195498	total: 5.87s	remaining: 11.8s
332:	learn: 0.4194467	total: 5.88s	remaining: 11.8s
333:	learn: 0.4193732	total: 5.89s	remaining: 11.7s
334:	learn: 0.4192776	total: 5.9s	remaining: 11.7s
335:	learn: 0.4191596	total: 5.91s	remaining: 11.7s
336:	learn: 0.4191455	total: 5.92s	remaining: 11.6s
337:	learn: 0.4190953	total: 5.92s	remaining: 11.6s
338:	learn: 0.4190346	total: 5.93s	remaining: 11.6s
339:	learn: 0.4190202	total: 5.94s	remaining: 11.5s
340:	learn: 0.4189149	total: 5.96s	remaining: 11.5s
341:	learn: 0.4188565	total: 5.96s	remaining: 11.5s
342:	learn: 0.4188006	total: 5.96s	remaining: 11.4s
343:	learn: 0.418

494:	learn: 0.4096559	total: 8.26s	remaining: 8.43s
495:	learn: 0.4095431	total: 8.3s	remaining: 8.44s
496:	learn: 0.4094573	total: 8.38s	remaining: 8.48s
497:	learn: 0.4093919	total: 8.38s	remaining: 8.45s
498:	learn: 0.4093589	total: 8.39s	remaining: 8.43s
499:	learn: 0.4093031	total: 8.4s	remaining: 8.4s
500:	learn: 0.4092633	total: 8.4s	remaining: 8.37s
501:	learn: 0.4092065	total: 8.45s	remaining: 8.38s
502:	learn: 0.4091603	total: 8.47s	remaining: 8.37s
503:	learn: 0.4090920	total: 8.49s	remaining: 8.36s
504:	learn: 0.4089820	total: 8.51s	remaining: 8.34s
505:	learn: 0.4089606	total: 8.52s	remaining: 8.32s
506:	learn: 0.4088637	total: 8.53s	remaining: 8.29s
507:	learn: 0.4087716	total: 8.54s	remaining: 8.27s
508:	learn: 0.4087164	total: 8.55s	remaining: 8.25s
509:	learn: 0.4086708	total: 8.56s	remaining: 8.22s
510:	learn: 0.4086242	total: 8.57s	remaining: 8.2s
511:	learn: 0.4085258	total: 8.61s	remaining: 8.21s
512:	learn: 0.4084950	total: 8.63s	remaining: 8.19s
513:	learn: 0.408

656:	learn: 0.4009962	total: 11s	remaining: 5.74s
657:	learn: 0.4009280	total: 11s	remaining: 5.71s
658:	learn: 0.4009209	total: 11s	remaining: 5.69s
659:	learn: 0.4008681	total: 11s	remaining: 5.67s
660:	learn: 0.4007840	total: 11s	remaining: 5.65s
661:	learn: 0.4007594	total: 11s	remaining: 5.63s
662:	learn: 0.4006949	total: 11s	remaining: 5.61s
663:	learn: 0.4006558	total: 11.1s	remaining: 5.59s
664:	learn: 0.4005869	total: 11.1s	remaining: 5.57s
665:	learn: 0.4005086	total: 11.1s	remaining: 5.55s
666:	learn: 0.4004558	total: 11.1s	remaining: 5.52s
667:	learn: 0.4003990	total: 11.2s	remaining: 5.54s
668:	learn: 0.4003786	total: 11.2s	remaining: 5.54s
669:	learn: 0.4002780	total: 11.2s	remaining: 5.51s
670:	learn: 0.4002650	total: 11.2s	remaining: 5.49s
671:	learn: 0.4002538	total: 11.2s	remaining: 5.47s
672:	learn: 0.4001888	total: 11.2s	remaining: 5.45s
673:	learn: 0.4001203	total: 11.2s	remaining: 5.43s
674:	learn: 0.4000889	total: 11.2s	remaining: 5.41s
675:	learn: 0.4000436	tota

822:	learn: 0.3936600	total: 14.3s	remaining: 3.07s
823:	learn: 0.3936262	total: 14.3s	remaining: 3.05s
824:	learn: 0.3935733	total: 14.3s	remaining: 3.03s
825:	learn: 0.3935722	total: 14.3s	remaining: 3.01s
826:	learn: 0.3935640	total: 14.3s	remaining: 2.99s
827:	learn: 0.3935475	total: 14.3s	remaining: 2.97s
828:	learn: 0.3935182	total: 14.3s	remaining: 2.96s
829:	learn: 0.3934988	total: 14.3s	remaining: 2.94s
830:	learn: 0.3934696	total: 14.4s	remaining: 2.92s
831:	learn: 0.3934393	total: 14.4s	remaining: 2.9s
832:	learn: 0.3934085	total: 14.4s	remaining: 2.88s
833:	learn: 0.3933792	total: 14.4s	remaining: 2.86s
834:	learn: 0.3933714	total: 14.4s	remaining: 2.84s
835:	learn: 0.3933586	total: 14.4s	remaining: 2.83s
836:	learn: 0.3933128	total: 14.4s	remaining: 2.81s
837:	learn: 0.3932881	total: 14.4s	remaining: 2.79s
838:	learn: 0.3932627	total: 14.4s	remaining: 2.77s
839:	learn: 0.3932560	total: 14.5s	remaining: 2.75s
840:	learn: 0.3932495	total: 14.5s	remaining: 2.73s
841:	learn: 0

991:	learn: 0.3880076	total: 17.4s	remaining: 140ms
992:	learn: 0.3879745	total: 17.4s	remaining: 122ms
993:	learn: 0.3879624	total: 17.4s	remaining: 105ms
994:	learn: 0.3878985	total: 17.4s	remaining: 87.6ms
995:	learn: 0.3878973	total: 17.4s	remaining: 70.1ms
996:	learn: 0.3878682	total: 17.5s	remaining: 52.5ms
997:	learn: 0.3878284	total: 17.5s	remaining: 35ms
998:	learn: 0.3877828	total: 17.5s	remaining: 17.5ms
999:	learn: 0.3877489	total: 17.5s	remaining: 0us


In [9]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.443056


In [10]:
model = CatBoostRegressor(iterations=1500, learning_rate=0.01,
                           depth=8,random_seed=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)

0:	learn: 0.4943723	total: 9.86ms	remaining: 14.8s
1:	learn: 0.4935655	total: 20.1ms	remaining: 15.1s
2:	learn: 0.4928174	total: 26.3ms	remaining: 13.1s
3:	learn: 0.4919774	total: 60ms	remaining: 22.4s
4:	learn: 0.4912367	total: 65.7ms	remaining: 19.6s
5:	learn: 0.4905298	total: 70.6ms	remaining: 17.6s
6:	learn: 0.4897530	total: 78.8ms	remaining: 16.8s
7:	learn: 0.4890813	total: 85.8ms	remaining: 16s
8:	learn: 0.4885026	total: 88.3ms	remaining: 14.6s
9:	learn: 0.4878939	total: 91.3ms	remaining: 13.6s
10:	learn: 0.4872443	total: 102ms	remaining: 13.8s
11:	learn: 0.4865641	total: 110ms	remaining: 13.7s
12:	learn: 0.4858869	total: 116ms	remaining: 13.2s
13:	learn: 0.4852112	total: 126ms	remaining: 13.4s
14:	learn: 0.4846531	total: 130ms	remaining: 12.9s
15:	learn: 0.4839934	total: 138ms	remaining: 12.8s
16:	learn: 0.4833707	total: 149ms	remaining: 13s
17:	learn: 0.4827203	total: 158ms	remaining: 13s
18:	learn: 0.4820611	total: 166ms	remaining: 12.9s
19:	learn: 0.4814906	total: 172ms	remai

174:	learn: 0.4447527	total: 1.91s	remaining: 14.5s
175:	learn: 0.4446820	total: 1.92s	remaining: 14.5s
176:	learn: 0.4446536	total: 1.93s	remaining: 14.4s
177:	learn: 0.4446166	total: 1.93s	remaining: 14.3s
178:	learn: 0.4444830	total: 1.94s	remaining: 14.3s
179:	learn: 0.4444103	total: 1.95s	remaining: 14.3s
180:	learn: 0.4443228	total: 1.96s	remaining: 14.3s
181:	learn: 0.4442101	total: 1.98s	remaining: 14.3s
182:	learn: 0.4441234	total: 1.99s	remaining: 14.3s
183:	learn: 0.4439693	total: 2s	remaining: 14.3s
184:	learn: 0.4438973	total: 2.01s	remaining: 14.3s
185:	learn: 0.4437991	total: 2.02s	remaining: 14.3s
186:	learn: 0.4436780	total: 2.03s	remaining: 14.3s
187:	learn: 0.4436047	total: 2.04s	remaining: 14.2s
188:	learn: 0.4434871	total: 2.05s	remaining: 14.2s
189:	learn: 0.4434254	total: 2.06s	remaining: 14.2s
190:	learn: 0.4433537	total: 2.07s	remaining: 14.2s
191:	learn: 0.4432822	total: 2.08s	remaining: 14.2s
192:	learn: 0.4432005	total: 2.1s	remaining: 14.2s
193:	learn: 0.44

336:	learn: 0.4355962	total: 4.35s	remaining: 15s
337:	learn: 0.4355596	total: 4.37s	remaining: 15s
338:	learn: 0.4355449	total: 4.38s	remaining: 15s
339:	learn: 0.4354911	total: 4.39s	remaining: 15s
340:	learn: 0.4354486	total: 4.39s	remaining: 14.9s
341:	learn: 0.4354294	total: 4.41s	remaining: 14.9s
342:	learn: 0.4353759	total: 4.42s	remaining: 14.9s
343:	learn: 0.4353428	total: 4.42s	remaining: 14.9s
344:	learn: 0.4353127	total: 4.44s	remaining: 14.9s
345:	learn: 0.4352895	total: 4.45s	remaining: 14.8s
346:	learn: 0.4352434	total: 4.45s	remaining: 14.8s
347:	learn: 0.4352383	total: 4.46s	remaining: 14.8s
348:	learn: 0.4352118	total: 4.47s	remaining: 14.7s
349:	learn: 0.4351350	total: 4.48s	remaining: 14.7s
350:	learn: 0.4350504	total: 4.48s	remaining: 14.7s
351:	learn: 0.4350084	total: 4.5s	remaining: 14.7s
352:	learn: 0.4349698	total: 4.5s	remaining: 14.6s
353:	learn: 0.4349048	total: 4.51s	remaining: 14.6s
354:	learn: 0.4348611	total: 4.53s	remaining: 14.6s
355:	learn: 0.4348257	

505:	learn: 0.4304025	total: 6.27s	remaining: 12.3s
506:	learn: 0.4303926	total: 6.29s	remaining: 12.3s
507:	learn: 0.4303799	total: 6.3s	remaining: 12.3s
508:	learn: 0.4303508	total: 6.32s	remaining: 12.3s
509:	learn: 0.4303145	total: 6.33s	remaining: 12.3s
510:	learn: 0.4303131	total: 6.34s	remaining: 12.3s
511:	learn: 0.4302997	total: 6.35s	remaining: 12.3s
512:	learn: 0.4302925	total: 6.37s	remaining: 12.3s
513:	learn: 0.4302843	total: 6.38s	remaining: 12.2s
514:	learn: 0.4302450	total: 6.39s	remaining: 12.2s
515:	learn: 0.4302191	total: 6.4s	remaining: 12.2s
516:	learn: 0.4301888	total: 6.41s	remaining: 12.2s
517:	learn: 0.4301574	total: 6.42s	remaining: 12.2s
518:	learn: 0.4301197	total: 6.43s	remaining: 12.2s
519:	learn: 0.4300894	total: 6.45s	remaining: 12.1s
520:	learn: 0.4300526	total: 6.47s	remaining: 12.2s
521:	learn: 0.4300199	total: 6.49s	remaining: 12.2s
522:	learn: 0.4299811	total: 6.5s	remaining: 12.1s
523:	learn: 0.4299674	total: 6.51s	remaining: 12.1s
524:	learn: 0.4

674:	learn: 0.4268027	total: 8.24s	remaining: 10.1s
675:	learn: 0.4267904	total: 8.26s	remaining: 10.1s
676:	learn: 0.4267528	total: 8.27s	remaining: 10.1s
677:	learn: 0.4267302	total: 8.28s	remaining: 10s
678:	learn: 0.4267001	total: 8.29s	remaining: 10s
679:	learn: 0.4266699	total: 8.3s	remaining: 10s
680:	learn: 0.4266629	total: 8.3s	remaining: 9.99s
681:	learn: 0.4266334	total: 8.32s	remaining: 9.98s
682:	learn: 0.4266074	total: 8.33s	remaining: 9.96s
683:	learn: 0.4265744	total: 8.38s	remaining: 9.99s
684:	learn: 0.4265448	total: 8.4s	remaining: 9.99s
685:	learn: 0.4265180	total: 8.42s	remaining: 9.99s
686:	learn: 0.4264982	total: 8.44s	remaining: 9.98s
687:	learn: 0.4264808	total: 8.45s	remaining: 9.97s
688:	learn: 0.4264487	total: 8.46s	remaining: 9.95s
689:	learn: 0.4264067	total: 8.47s	remaining: 9.94s
690:	learn: 0.4263661	total: 8.48s	remaining: 9.93s
691:	learn: 0.4263401	total: 8.49s	remaining: 9.91s
692:	learn: 0.4262903	total: 8.5s	remaining: 9.9s
693:	learn: 0.4262752	t

837:	learn: 0.4233359	total: 10.4s	remaining: 8.23s
838:	learn: 0.4233128	total: 10.4s	remaining: 8.23s
839:	learn: 0.4232961	total: 10.5s	remaining: 8.22s
840:	learn: 0.4232833	total: 10.5s	remaining: 8.21s
841:	learn: 0.4232549	total: 10.5s	remaining: 8.2s
842:	learn: 0.4232417	total: 10.5s	remaining: 8.19s
843:	learn: 0.4232282	total: 10.5s	remaining: 8.17s
844:	learn: 0.4231914	total: 10.5s	remaining: 8.16s
845:	learn: 0.4231644	total: 10.5s	remaining: 8.14s
846:	learn: 0.4231369	total: 10.5s	remaining: 8.13s
847:	learn: 0.4230853	total: 10.6s	remaining: 8.11s
848:	learn: 0.4230618	total: 10.6s	remaining: 8.1s
849:	learn: 0.4230236	total: 10.6s	remaining: 8.08s
850:	learn: 0.4229852	total: 10.6s	remaining: 8.07s
851:	learn: 0.4229677	total: 10.6s	remaining: 8.06s
852:	learn: 0.4229482	total: 10.6s	remaining: 8.05s
853:	learn: 0.4229131	total: 10.6s	remaining: 8.04s
854:	learn: 0.4228930	total: 10.6s	remaining: 8.02s
855:	learn: 0.4228737	total: 10.6s	remaining: 8s
856:	learn: 0.422

1009:	learn: 0.4189795	total: 12.4s	remaining: 6.01s
1010:	learn: 0.4189710	total: 12.4s	remaining: 6s
1011:	learn: 0.4189486	total: 12.4s	remaining: 5.99s
1012:	learn: 0.4189326	total: 12.4s	remaining: 5.97s
1013:	learn: 0.4188919	total: 12.4s	remaining: 5.96s
1014:	learn: 0.4188674	total: 12.4s	remaining: 5.95s
1015:	learn: 0.4188146	total: 12.5s	remaining: 5.93s
1016:	learn: 0.4187934	total: 12.5s	remaining: 5.92s
1017:	learn: 0.4187767	total: 12.5s	remaining: 5.91s
1018:	learn: 0.4187486	total: 12.5s	remaining: 5.9s
1019:	learn: 0.4187321	total: 12.5s	remaining: 5.88s
1020:	learn: 0.4186946	total: 12.5s	remaining: 5.87s
1021:	learn: 0.4186663	total: 12.5s	remaining: 5.86s
1022:	learn: 0.4186363	total: 12.6s	remaining: 5.85s
1023:	learn: 0.4186308	total: 12.6s	remaining: 5.84s
1024:	learn: 0.4185884	total: 12.6s	remaining: 5.83s
1025:	learn: 0.4185568	total: 12.6s	remaining: 5.81s
1026:	learn: 0.4185511	total: 12.6s	remaining: 5.8s
1027:	learn: 0.4185206	total: 12.6s	remaining: 5.79

1167:	learn: 0.4149845	total: 14.5s	remaining: 4.13s
1168:	learn: 0.4149594	total: 14.5s	remaining: 4.12s
1169:	learn: 0.4149276	total: 14.6s	remaining: 4.11s
1170:	learn: 0.4149048	total: 14.6s	remaining: 4.09s
1171:	learn: 0.4148954	total: 14.6s	remaining: 4.08s
1172:	learn: 0.4148819	total: 14.6s	remaining: 4.07s
1173:	learn: 0.4148561	total: 14.6s	remaining: 4.05s
1174:	learn: 0.4148274	total: 14.6s	remaining: 4.04s
1175:	learn: 0.4147874	total: 14.6s	remaining: 4.03s
1176:	learn: 0.4147702	total: 14.6s	remaining: 4.02s
1177:	learn: 0.4147490	total: 14.6s	remaining: 4s
1178:	learn: 0.4147222	total: 14.7s	remaining: 3.99s
1179:	learn: 0.4147136	total: 14.7s	remaining: 3.98s
1180:	learn: 0.4146681	total: 14.7s	remaining: 3.96s
1181:	learn: 0.4146510	total: 14.7s	remaining: 3.95s
1182:	learn: 0.4146327	total: 14.7s	remaining: 3.94s
1183:	learn: 0.4146149	total: 14.7s	remaining: 3.93s
1184:	learn: 0.4145796	total: 14.7s	remaining: 3.91s
1185:	learn: 0.4145635	total: 14.7s	remaining: 3.

1333:	learn: 0.4111712	total: 16.4s	remaining: 2.04s
1334:	learn: 0.4111495	total: 16.4s	remaining: 2.03s
1335:	learn: 0.4111372	total: 16.5s	remaining: 2.02s
1336:	learn: 0.4111113	total: 16.5s	remaining: 2.01s
1337:	learn: 0.4110851	total: 16.5s	remaining: 2s
1338:	learn: 0.4110584	total: 16.5s	remaining: 1.98s
1339:	learn: 0.4110398	total: 16.5s	remaining: 1.97s
1340:	learn: 0.4110096	total: 16.5s	remaining: 1.96s
1341:	learn: 0.4109706	total: 16.5s	remaining: 1.95s
1342:	learn: 0.4109563	total: 16.6s	remaining: 1.94s
1343:	learn: 0.4109444	total: 16.6s	remaining: 1.92s
1344:	learn: 0.4109091	total: 16.6s	remaining: 1.91s
1345:	learn: 0.4108781	total: 16.6s	remaining: 1.9s
1346:	learn: 0.4108671	total: 16.6s	remaining: 1.89s
1347:	learn: 0.4108383	total: 16.6s	remaining: 1.87s
1348:	learn: 0.4108115	total: 16.6s	remaining: 1.86s
1349:	learn: 0.4107829	total: 16.7s	remaining: 1.85s
1350:	learn: 0.4107361	total: 16.7s	remaining: 1.84s
1351:	learn: 0.4107217	total: 16.7s	remaining: 1.8

1493:	learn: 0.4077290	total: 18.3s	remaining: 73.5ms
1494:	learn: 0.4077101	total: 18.3s	remaining: 61.3ms
1495:	learn: 0.4076900	total: 18.3s	remaining: 49ms
1496:	learn: 0.4076707	total: 18.4s	remaining: 36.8ms
1497:	learn: 0.4076703	total: 18.4s	remaining: 24.5ms
1498:	learn: 0.4076552	total: 18.4s	remaining: 12.3ms
1499:	learn: 0.4076377	total: 18.4s	remaining: 0us


In [11]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.440091


In [12]:
#Catboost como problema de clasificación
model = CatBoostClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)

Learning rate set to 0.022283
0:	learn: 0.6880089	total: 83.4ms	remaining: 1m 23s
1:	learn: 0.6836855	total: 96.9ms	remaining: 48.4s
2:	learn: 0.6803530	total: 113ms	remaining: 37.7s
3:	learn: 0.6761401	total: 122ms	remaining: 30.3s
4:	learn: 0.6727508	total: 132ms	remaining: 26.4s
5:	learn: 0.6686969	total: 141ms	remaining: 23.4s
6:	learn: 0.6654637	total: 151ms	remaining: 21.5s
7:	learn: 0.6616538	total: 164ms	remaining: 20.3s
8:	learn: 0.6579594	total: 176ms	remaining: 19.4s
9:	learn: 0.6546455	total: 185ms	remaining: 18.4s
10:	learn: 0.6517560	total: 194ms	remaining: 17.4s
11:	learn: 0.6485809	total: 203ms	remaining: 16.7s
12:	learn: 0.6461476	total: 210ms	remaining: 16s
13:	learn: 0.6434997	total: 223ms	remaining: 15.7s
14:	learn: 0.6413066	total: 233ms	remaining: 15.3s
15:	learn: 0.6388462	total: 241ms	remaining: 14.8s
16:	learn: 0.6368645	total: 257ms	remaining: 14.8s
17:	learn: 0.6344731	total: 267ms	remaining: 14.6s
18:	learn: 0.6323927	total: 274ms	remaining: 14.2s
19:	learn:

165:	learn: 0.5643141	total: 1.92s	remaining: 9.66s
166:	learn: 0.5643051	total: 1.94s	remaining: 9.7s
167:	learn: 0.5642168	total: 1.96s	remaining: 9.68s
168:	learn: 0.5641531	total: 1.97s	remaining: 9.68s
169:	learn: 0.5639856	total: 2s	remaining: 9.78s
170:	learn: 0.5638480	total: 2.02s	remaining: 9.82s
171:	learn: 0.5638157	total: 2.03s	remaining: 9.78s
172:	learn: 0.5636232	total: 2.04s	remaining: 9.75s
173:	learn: 0.5634970	total: 2.05s	remaining: 9.73s
174:	learn: 0.5633742	total: 2.06s	remaining: 9.71s
175:	learn: 0.5632414	total: 2.07s	remaining: 9.69s
176:	learn: 0.5632148	total: 2.08s	remaining: 9.65s
177:	learn: 0.5631137	total: 2.09s	remaining: 9.64s
178:	learn: 0.5629819	total: 2.1s	remaining: 9.63s
179:	learn: 0.5629299	total: 2.11s	remaining: 9.62s
180:	learn: 0.5628415	total: 2.12s	remaining: 9.6s
181:	learn: 0.5627568	total: 2.14s	remaining: 9.6s
182:	learn: 0.5626486	total: 2.15s	remaining: 9.59s
183:	learn: 0.5625859	total: 2.16s	remaining: 9.58s
184:	learn: 0.56247

324:	learn: 0.5488821	total: 3.68s	remaining: 7.63s
325:	learn: 0.5488453	total: 3.7s	remaining: 7.65s
326:	learn: 0.5487767	total: 3.71s	remaining: 7.63s
327:	learn: 0.5486636	total: 3.72s	remaining: 7.62s
328:	learn: 0.5485991	total: 3.73s	remaining: 7.6s
329:	learn: 0.5485884	total: 3.73s	remaining: 7.58s
330:	learn: 0.5484183	total: 3.74s	remaining: 7.57s
331:	learn: 0.5482424	total: 3.75s	remaining: 7.54s
332:	learn: 0.5481285	total: 3.76s	remaining: 7.53s
333:	learn: 0.5479747	total: 3.77s	remaining: 7.51s
334:	learn: 0.5478800	total: 3.78s	remaining: 7.5s
335:	learn: 0.5478699	total: 3.78s	remaining: 7.47s
336:	learn: 0.5477253	total: 3.79s	remaining: 7.46s
337:	learn: 0.5475954	total: 3.8s	remaining: 7.45s
338:	learn: 0.5474599	total: 3.81s	remaining: 7.43s
339:	learn: 0.5473608	total: 3.82s	remaining: 7.42s
340:	learn: 0.5471765	total: 3.83s	remaining: 7.4s
341:	learn: 0.5470715	total: 3.84s	remaining: 7.38s
342:	learn: 0.5470312	total: 3.85s	remaining: 7.37s
343:	learn: 0.546

494:	learn: 0.5321928	total: 5.44s	remaining: 5.55s
495:	learn: 0.5321022	total: 5.45s	remaining: 5.53s
496:	learn: 0.5320491	total: 5.46s	remaining: 5.52s
497:	learn: 0.5318897	total: 5.47s	remaining: 5.52s
498:	learn: 0.5317929	total: 5.49s	remaining: 5.51s
499:	learn: 0.5316527	total: 5.5s	remaining: 5.5s
500:	learn: 0.5315535	total: 5.51s	remaining: 5.49s
501:	learn: 0.5314539	total: 5.52s	remaining: 5.48s
502:	learn: 0.5313549	total: 5.53s	remaining: 5.46s
503:	learn: 0.5312502	total: 5.54s	remaining: 5.45s
504:	learn: 0.5310753	total: 5.55s	remaining: 5.44s
505:	learn: 0.5309032	total: 5.55s	remaining: 5.42s
506:	learn: 0.5307027	total: 5.56s	remaining: 5.41s
507:	learn: 0.5305263	total: 5.57s	remaining: 5.39s
508:	learn: 0.5304473	total: 5.58s	remaining: 5.38s
509:	learn: 0.5302861	total: 5.59s	remaining: 5.37s
510:	learn: 0.5301837	total: 5.6s	remaining: 5.36s
511:	learn: 0.5300533	total: 5.61s	remaining: 5.35s
512:	learn: 0.5299875	total: 5.63s	remaining: 5.34s
513:	learn: 0.5

657:	learn: 0.5161544	total: 7.38s	remaining: 3.84s
658:	learn: 0.5160422	total: 7.39s	remaining: 3.83s
659:	learn: 0.5159529	total: 7.41s	remaining: 3.81s
660:	learn: 0.5158734	total: 7.41s	remaining: 3.8s
661:	learn: 0.5158135	total: 7.43s	remaining: 3.79s
662:	learn: 0.5157661	total: 7.43s	remaining: 3.78s
663:	learn: 0.5157095	total: 7.44s	remaining: 3.77s
664:	learn: 0.5157069	total: 7.45s	remaining: 3.75s
665:	learn: 0.5156157	total: 7.46s	remaining: 3.74s
666:	learn: 0.5154926	total: 7.47s	remaining: 3.73s
667:	learn: 0.5153979	total: 7.48s	remaining: 3.72s
668:	learn: 0.5153471	total: 7.49s	remaining: 3.71s
669:	learn: 0.5152789	total: 7.5s	remaining: 3.69s
670:	learn: 0.5151479	total: 7.51s	remaining: 3.68s
671:	learn: 0.5150002	total: 7.52s	remaining: 3.67s
672:	learn: 0.5149541	total: 7.53s	remaining: 3.66s
673:	learn: 0.5149092	total: 7.53s	remaining: 3.64s
674:	learn: 0.5148807	total: 7.54s	remaining: 3.63s
675:	learn: 0.5147812	total: 7.55s	remaining: 3.62s
676:	learn: 0.

824:	learn: 0.5031081	total: 9.08s	remaining: 1.93s
825:	learn: 0.5030716	total: 9.09s	remaining: 1.91s
826:	learn: 0.5030467	total: 9.11s	remaining: 1.91s
827:	learn: 0.5029307	total: 9.15s	remaining: 1.9s
828:	learn: 0.5028640	total: 9.19s	remaining: 1.9s
829:	learn: 0.5027986	total: 9.2s	remaining: 1.88s
830:	learn: 0.5027975	total: 9.21s	remaining: 1.87s
831:	learn: 0.5027555	total: 9.22s	remaining: 1.86s
832:	learn: 0.5027002	total: 9.22s	remaining: 1.85s
833:	learn: 0.5025620	total: 9.23s	remaining: 1.84s
834:	learn: 0.5024786	total: 9.24s	remaining: 1.83s
835:	learn: 0.5023886	total: 9.25s	remaining: 1.81s
836:	learn: 0.5023414	total: 9.26s	remaining: 1.8s
837:	learn: 0.5022741	total: 9.28s	remaining: 1.79s
838:	learn: 0.5022011	total: 9.29s	remaining: 1.78s
839:	learn: 0.5021496	total: 9.3s	remaining: 1.77s
840:	learn: 0.5020853	total: 9.3s	remaining: 1.76s
841:	learn: 0.5020264	total: 9.32s	remaining: 1.75s
842:	learn: 0.5020195	total: 9.33s	remaining: 1.74s
843:	learn: 0.5019

996:	learn: 0.4915782	total: 10.9s	remaining: 32.7ms
997:	learn: 0.4914877	total: 10.9s	remaining: 21.9ms
998:	learn: 0.4914198	total: 10.9s	remaining: 10.9ms
999:	learn: 0.4914091	total: 10.9s	remaining: 0us


In [13]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

f1score = f1_score(y_test, preds)
print(f'Counts model score: {f1score*100}%')

RMSE: 0.544175
Counts model score: 61.61702127659575%


In [14]:
model = CatBoostClassifier(iterations=1500, learning_rate=0.01,
                           depth=8,random_seed=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)

0:	learn: 0.6911514	total: 16.3ms	remaining: 24.4s
1:	learn: 0.6889364	total: 30.7ms	remaining: 23s
2:	learn: 0.6872120	total: 37.7ms	remaining: 18.8s
3:	learn: 0.6846854	total: 49.8ms	remaining: 18.6s
4:	learn: 0.6824243	total: 64.6ms	remaining: 19.3s
5:	learn: 0.6802184	total: 80.3ms	remaining: 20s
6:	learn: 0.6783771	total: 89.7ms	remaining: 19.1s
7:	learn: 0.6766257	total: 99.4ms	remaining: 18.5s
8:	learn: 0.6746611	total: 114ms	remaining: 18.9s
9:	learn: 0.6728067	total: 125ms	remaining: 18.6s
10:	learn: 0.6711691	total: 142ms	remaining: 19.3s
11:	learn: 0.6694595	total: 155ms	remaining: 19.2s
12:	learn: 0.6674479	total: 171ms	remaining: 19.6s
13:	learn: 0.6655561	total: 205ms	remaining: 21.7s
14:	learn: 0.6638939	total: 225ms	remaining: 22.3s
15:	learn: 0.6619557	total: 255ms	remaining: 23.7s
16:	learn: 0.6606373	total: 263ms	remaining: 23s
17:	learn: 0.6590829	total: 271ms	remaining: 22.3s
18:	learn: 0.6579777	total: 277ms	remaining: 21.6s
19:	learn: 0.6562607	total: 290ms	remai

173:	learn: 0.5726960	total: 2.74s	remaining: 20.9s
174:	learn: 0.5724949	total: 2.76s	remaining: 20.9s
175:	learn: 0.5722778	total: 2.77s	remaining: 20.8s
176:	learn: 0.5721570	total: 2.78s	remaining: 20.8s
177:	learn: 0.5720398	total: 2.79s	remaining: 20.7s
178:	learn: 0.5717694	total: 2.81s	remaining: 20.7s
179:	learn: 0.5715654	total: 2.82s	remaining: 20.7s
180:	learn: 0.5714270	total: 2.84s	remaining: 20.7s
181:	learn: 0.5712519	total: 2.85s	remaining: 20.6s
182:	learn: 0.5710944	total: 2.86s	remaining: 20.6s
183:	learn: 0.5708280	total: 2.88s	remaining: 20.6s
184:	learn: 0.5706201	total: 2.9s	remaining: 20.7s
185:	learn: 0.5703945	total: 2.93s	remaining: 20.7s
186:	learn: 0.5702147	total: 2.95s	remaining: 20.7s
187:	learn: 0.5698673	total: 2.96s	remaining: 20.7s
188:	learn: 0.5696377	total: 2.97s	remaining: 20.6s
189:	learn: 0.5695338	total: 2.98s	remaining: 20.6s
190:	learn: 0.5693772	total: 3s	remaining: 20.5s
191:	learn: 0.5692193	total: 3.01s	remaining: 20.5s
192:	learn: 0.56

343:	learn: 0.5509173	total: 5.64s	remaining: 19s
344:	learn: 0.5508516	total: 5.66s	remaining: 18.9s
345:	learn: 0.5508165	total: 5.67s	remaining: 18.9s
346:	learn: 0.5507692	total: 5.68s	remaining: 18.9s
347:	learn: 0.5507299	total: 5.7s	remaining: 18.9s
348:	learn: 0.5505957	total: 5.71s	remaining: 18.8s
349:	learn: 0.5504755	total: 5.73s	remaining: 18.8s
350:	learn: 0.5503719	total: 5.74s	remaining: 18.8s
351:	learn: 0.5501777	total: 5.75s	remaining: 18.8s
352:	learn: 0.5501156	total: 5.77s	remaining: 18.7s
353:	learn: 0.5499794	total: 5.78s	remaining: 18.7s
354:	learn: 0.5498912	total: 5.79s	remaining: 18.7s
355:	learn: 0.5498732	total: 5.8s	remaining: 18.6s
356:	learn: 0.5498252	total: 5.81s	remaining: 18.6s
357:	learn: 0.5496805	total: 5.83s	remaining: 18.6s
358:	learn: 0.5495254	total: 5.84s	remaining: 18.6s
359:	learn: 0.5493900	total: 5.86s	remaining: 18.6s
360:	learn: 0.5493180	total: 5.88s	remaining: 18.5s
361:	learn: 0.5492843	total: 5.89s	remaining: 18.5s
362:	learn: 0.54

512:	learn: 0.5384606	total: 8.37s	remaining: 16.1s
513:	learn: 0.5383587	total: 8.39s	remaining: 16.1s
514:	learn: 0.5383094	total: 8.4s	remaining: 16.1s
515:	learn: 0.5383014	total: 8.41s	remaining: 16s
516:	learn: 0.5382634	total: 8.43s	remaining: 16s
517:	learn: 0.5382298	total: 8.44s	remaining: 16s
518:	learn: 0.5381883	total: 8.46s	remaining: 16s
519:	learn: 0.5381108	total: 8.47s	remaining: 16s
520:	learn: 0.5379951	total: 8.49s	remaining: 16s
521:	learn: 0.5379281	total: 8.51s	remaining: 15.9s
522:	learn: 0.5379025	total: 8.53s	remaining: 15.9s
523:	learn: 0.5378314	total: 8.56s	remaining: 15.9s
524:	learn: 0.5376430	total: 8.59s	remaining: 15.9s
525:	learn: 0.5375464	total: 8.6s	remaining: 15.9s
526:	learn: 0.5375303	total: 8.65s	remaining: 16s
527:	learn: 0.5374152	total: 8.67s	remaining: 16s
528:	learn: 0.5373118	total: 8.68s	remaining: 15.9s
529:	learn: 0.5372728	total: 8.7s	remaining: 15.9s
530:	learn: 0.5372223	total: 8.71s	remaining: 15.9s
531:	learn: 0.5371719	total: 8.

671:	learn: 0.5289392	total: 11.5s	remaining: 14.1s
672:	learn: 0.5288960	total: 11.5s	remaining: 14.1s
673:	learn: 0.5288777	total: 11.5s	remaining: 14.1s
674:	learn: 0.5288092	total: 11.5s	remaining: 14.1s
675:	learn: 0.5286955	total: 11.5s	remaining: 14.1s
676:	learn: 0.5286335	total: 11.5s	remaining: 14s
677:	learn: 0.5286159	total: 11.6s	remaining: 14s
678:	learn: 0.5285549	total: 11.6s	remaining: 14s
679:	learn: 0.5285363	total: 11.6s	remaining: 14s
680:	learn: 0.5284043	total: 11.6s	remaining: 13.9s
681:	learn: 0.5283632	total: 11.6s	remaining: 13.9s
682:	learn: 0.5283355	total: 11.6s	remaining: 13.9s
683:	learn: 0.5283002	total: 11.6s	remaining: 13.9s
684:	learn: 0.5282098	total: 11.7s	remaining: 13.9s
685:	learn: 0.5281579	total: 11.7s	remaining: 13.9s
686:	learn: 0.5280958	total: 11.7s	remaining: 13.8s
687:	learn: 0.5280478	total: 11.7s	remaining: 13.8s
688:	learn: 0.5279788	total: 11.7s	remaining: 13.8s
689:	learn: 0.5279453	total: 11.7s	remaining: 13.8s
690:	learn: 0.527836

839:	learn: 0.5191386	total: 14.4s	remaining: 11.3s
840:	learn: 0.5191165	total: 14.4s	remaining: 11.3s
841:	learn: 0.5190396	total: 14.4s	remaining: 11.3s
842:	learn: 0.5190065	total: 14.4s	remaining: 11.2s
843:	learn: 0.5188392	total: 14.4s	remaining: 11.2s
844:	learn: 0.5187938	total: 14.5s	remaining: 11.2s
845:	learn: 0.5187651	total: 14.5s	remaining: 11.2s
846:	learn: 0.5187327	total: 14.5s	remaining: 11.2s
847:	learn: 0.5186209	total: 14.5s	remaining: 11.1s
848:	learn: 0.5185403	total: 14.5s	remaining: 11.1s
849:	learn: 0.5185181	total: 14.5s	remaining: 11.1s
850:	learn: 0.5184011	total: 14.5s	remaining: 11.1s
851:	learn: 0.5183743	total: 14.6s	remaining: 11.1s
852:	learn: 0.5183357	total: 14.6s	remaining: 11s
853:	learn: 0.5182442	total: 14.6s	remaining: 11s
854:	learn: 0.5181176	total: 14.6s	remaining: 11s
855:	learn: 0.5180743	total: 14.6s	remaining: 11s
856:	learn: 0.5180425	total: 14.6s	remaining: 11s
857:	learn: 0.5178961	total: 14.6s	remaining: 11s
858:	learn: 0.5177891	to

1007:	learn: 0.5066021	total: 16.9s	remaining: 8.27s
1008:	learn: 0.5065197	total: 17s	remaining: 8.27s
1009:	learn: 0.5064493	total: 17s	remaining: 8.27s
1010:	learn: 0.5064030	total: 17.1s	remaining: 8.26s
1011:	learn: 0.5063356	total: 17.1s	remaining: 8.24s
1012:	learn: 0.5062947	total: 17.1s	remaining: 8.22s
1013:	learn: 0.5061688	total: 17.1s	remaining: 8.2s
1014:	learn: 0.5060739	total: 17.1s	remaining: 8.19s
1015:	learn: 0.5059174	total: 17.1s	remaining: 8.17s
1016:	learn: 0.5058051	total: 17.2s	remaining: 8.15s
1017:	learn: 0.5057123	total: 17.2s	remaining: 8.13s
1018:	learn: 0.5056248	total: 17.2s	remaining: 8.11s
1019:	learn: 0.5055490	total: 17.2s	remaining: 8.09s
1020:	learn: 0.5054943	total: 17.2s	remaining: 8.08s
1021:	learn: 0.5054112	total: 17.2s	remaining: 8.06s
1022:	learn: 0.5053043	total: 17.2s	remaining: 8.04s
1023:	learn: 0.5052437	total: 17.3s	remaining: 8.02s
1024:	learn: 0.5051644	total: 17.3s	remaining: 8s
1025:	learn: 0.5050758	total: 17.3s	remaining: 7.98s
1

1164:	learn: 0.4949656	total: 19.5s	remaining: 5.61s
1165:	learn: 0.4948829	total: 19.5s	remaining: 5.59s
1166:	learn: 0.4948178	total: 19.6s	remaining: 5.58s
1167:	learn: 0.4947672	total: 19.6s	remaining: 5.56s
1168:	learn: 0.4947424	total: 19.6s	remaining: 5.54s
1169:	learn: 0.4946974	total: 19.6s	remaining: 5.53s
1170:	learn: 0.4946379	total: 19.6s	remaining: 5.51s
1171:	learn: 0.4945921	total: 19.6s	remaining: 5.49s
1172:	learn: 0.4945251	total: 19.6s	remaining: 5.47s
1173:	learn: 0.4944583	total: 19.7s	remaining: 5.46s
1174:	learn: 0.4943342	total: 19.7s	remaining: 5.44s
1175:	learn: 0.4942862	total: 19.7s	remaining: 5.42s
1176:	learn: 0.4941909	total: 19.7s	remaining: 5.41s
1177:	learn: 0.4941240	total: 19.7s	remaining: 5.39s
1178:	learn: 0.4940730	total: 19.7s	remaining: 5.37s
1179:	learn: 0.4940449	total: 19.8s	remaining: 5.36s
1180:	learn: 0.4939588	total: 19.8s	remaining: 5.34s
1181:	learn: 0.4939075	total: 19.8s	remaining: 5.32s
1182:	learn: 0.4938302	total: 19.8s	remaining:

1325:	learn: 0.4848333	total: 22.1s	remaining: 2.9s
1326:	learn: 0.4847563	total: 22.1s	remaining: 2.88s
1327:	learn: 0.4846940	total: 22.1s	remaining: 2.86s
1328:	learn: 0.4846495	total: 22.1s	remaining: 2.85s
1329:	learn: 0.4845912	total: 22.2s	remaining: 2.83s
1330:	learn: 0.4844947	total: 22.2s	remaining: 2.82s
1331:	learn: 0.4844210	total: 22.2s	remaining: 2.8s
1332:	learn: 0.4843422	total: 22.2s	remaining: 2.78s
1333:	learn: 0.4843225	total: 22.2s	remaining: 2.77s
1334:	learn: 0.4842594	total: 22.2s	remaining: 2.75s
1335:	learn: 0.4841976	total: 22.3s	remaining: 2.73s
1336:	learn: 0.4841583	total: 22.3s	remaining: 2.72s
1337:	learn: 0.4841314	total: 22.3s	remaining: 2.7s
1338:	learn: 0.4840621	total: 22.3s	remaining: 2.68s
1339:	learn: 0.4840357	total: 22.3s	remaining: 2.67s
1340:	learn: 0.4839609	total: 22.3s	remaining: 2.65s
1341:	learn: 0.4838597	total: 22.4s	remaining: 2.63s
1342:	learn: 0.4838033	total: 22.4s	remaining: 2.62s
1343:	learn: 0.4837842	total: 22.4s	remaining: 2.

1481:	learn: 0.4757943	total: 24.4s	remaining: 297ms
1482:	learn: 0.4756850	total: 24.5s	remaining: 280ms
1483:	learn: 0.4756382	total: 24.5s	remaining: 264ms
1484:	learn: 0.4755911	total: 24.5s	remaining: 247ms
1485:	learn: 0.4755541	total: 24.5s	remaining: 231ms
1486:	learn: 0.4754829	total: 24.5s	remaining: 214ms
1487:	learn: 0.4754287	total: 24.5s	remaining: 198ms
1488:	learn: 0.4753814	total: 24.5s	remaining: 181ms
1489:	learn: 0.4753235	total: 24.6s	remaining: 165ms
1490:	learn: 0.4752226	total: 24.6s	remaining: 148ms
1491:	learn: 0.4751328	total: 24.6s	remaining: 132ms
1492:	learn: 0.4750861	total: 24.6s	remaining: 115ms
1493:	learn: 0.4750187	total: 24.6s	remaining: 98.9ms
1494:	learn: 0.4749189	total: 24.6s	remaining: 82.4ms
1495:	learn: 0.4748674	total: 24.7s	remaining: 65.9ms
1496:	learn: 0.4748465	total: 24.7s	remaining: 49.4ms
1497:	learn: 0.4747863	total: 24.7s	remaining: 33ms
1498:	learn: 0.4747344	total: 24.7s	remaining: 16.5ms
1499:	learn: 0.4746541	total: 24.7s	remain

In [15]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

f1score = f1_score(y_test, preds)
print(f'Counts model score: {f1score*100}%')

RMSE: 0.536886
Counts model score: 62.828111769686714%
